In [1]:
from scipy.io import loadmat
import pandas as pd
import numpy as np
from xgboost import XGBClassifier
#from xgboost import predict_proba
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error  
import xgboost as xgb
from sklearn.model_selection import GridSearchCV #网格搜索
from sklearn.ensemble import GradientBoostingClassifier
from sklearn import ensemble
from sklearn.model_selection import train_test_split                 
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
import torch.utils.data as Data
from sklearn import preprocessing

In [2]:
m = loadmat("ADNI.mat") 

AD = m['AD']
NC = m['NC']

AD = pd.DataFrame(AD)
NC = pd.DataFrame(NC)

AD['label'] = [0]*51
NC['label'] = [1]*52

df = AD.append(NC)
label = df['label']
df=df.drop(['label'],axis = 1)

AD=AD.drop(['label'],axis = 1)
NC=NC.drop(['label'],axis = 1)

In [3]:
mm_scaler = preprocessing.MinMaxScaler()
df = mm_scaler.fit_transform(df)
df = pd.DataFrame(df)

x_train,x_test,y_train,y_test = train_test_split(df , label, test_size = 0.3 )  

In [4]:
import torch
from torch import nn
from torch.nn import init
import numpy as np
import sys
import matplotlib.pyplot as plt
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim

print(torch.__version__)

1.9.0


In [5]:
train_dataset = Data.TensorDataset(torch.tensor(x_train.values), torch.tensor(y_train.values))
test_dataset = Data.TensorDataset(torch.tensor(x_test.values), torch.tensor(y_test.values))
batch_size = 256
num_workers = 0
train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)

In [6]:
import time

num_inputs = 186
num_outputs = 2
hidden_dim1=32

#定义模型（添加）
class LinearNet(nn.Module):
    def __init__(self, num_inputs, num_outputs):
        super(LinearNet,self).__init__()
        #self.flatten = d2l.FlattenLayer()
        self.linear1 = nn.Linear(num_inputs,hidden_dim1)
        self.relu=nn.ReLU()
        self.batchNorm1d1 = nn.BatchNorm1d(hidden_dim1)
        self.linear2 = nn.Linear(hidden_dim1, num_outputs)
        self.dropout = nn.Dropout(p=0.2)  # dropout训练
    def init_weights(self):
        self.linear.weight.data.uniform_(-1, 1)
        self.linear.bias.data.fill_(0)
    def forward(self, x): # x shape: (batch, 1, 28, 28)
        y = self.linear1(x.view(x.shape[0],-1))
        #y = self.batchNorm1d1(y)
        y = self.relu(y)
        y = self.dropout(y)
        y = self.linear2(y)
        #y = self.batchNorm1d2(y)
        #y = self.relu(y)
        return y
    
net = LinearNet(num_inputs,num_outputs)
#for params in net.parameters(): 
#    init.normal_(params, mean=0, std=0.01)   #参数初始化
print(net) 

LinearNet(
  (linear1): Linear(in_features=186, out_features=32, bias=True)
  (relu): ReLU()
  (batchNorm1d1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=32, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)


In [7]:
def evaluate_accuracy(data_iter, net, device=None):
    if device is None and isinstance(net, torch.nn.Module):
        # 如果没指定device就使用net的device
        device = list(net.parameters())[0].device 
    acc_sum, n = 0.0, 0
    with torch.no_grad():
        for X, y in data_iter:
            if isinstance(net, torch.nn.Module):
                pred = net(X.to(device).to(torch.float32)).argmax(dim=1)
                net.eval() # 评估模式, 这会关闭dropout
                acc_sum += (net(X.to(device).to(torch.float32)).argmax(dim=1) == y.to(device).to(torch.float32)).float().sum().cpu().item()
                net.train() # 改回训练模式
            else: # 自定义的模型, 3.13节之后不会用到, 不考虑GPU
                if('is_training' in net.__code__.co_varnames): # 如果有is_training这个参数
                    # 将is_training设置成False
                    acc_sum += (net(X, is_training=False).argmax(dim=1) == y).float().sum().item() 
                else:
                    acc_sum += (net(X).argmax(dim=1) == y).float().sum().item() 
            n += y.shape[0]
    return acc_sum / n , pred

In [8]:
def all_results(pred,y):
    from sklearn.metrics import accuracy_score,roc_auc_score,auc,confusion_matrix,roc_curve
    fpr,tpr,threshold = roc_curve(y, pred) 
    roc_auc= auc(fpr,tpr)

    if(len(set(y))!=1):
        roc=roc_auc_score(y, pred)
        sensitivity=confusion_matrix(y, pred)[0][0]/(confusion_matrix(y, pred)[0][0]+confusion_matrix(y, pred)[0][1])
        specificity=confusion_matrix(y, pred)[1][1]/(confusion_matrix(y, pred)[1][1]+confusion_matrix(y, pred)[1][0])
    else:
        roc=0
        sensitivity=0
        specificity=0
    return roc,sensitivity,specificity

In [9]:
import time
from sklearn.metrics import accuracy_score,roc_auc_score,auc,confusion_matrix,roc_curve

net = LinearNet(num_inputs,num_outputs)
#for params in net.parameters(): 
#    init.normal_(params, mean=0, std=0.01)   #参数初始化
print(net) 

for X, y in test_iter:
    y_true = y
#roc,sensitivity,specificity = []

    
num_epochs = 100
now_time = time.time()
losses = []
accuracies = []

plot_test = []
plot_train = []
plot_roc = []
plot_sensitivity = []
plot_specificity = []
#损失函数和优化器
loss = torch.nn.CrossEntropyLoss()
LR = 0.1
optimizer = torch.optim.SGD(
                            net.parameters(), 
                            lr = LR,
                            weight_decay = 0.05
                             )

for epoch in range(num_epochs):
    
    if epoch % 10 == 0:
        for p in optimizer.param_groups:
            p['lr'] *= 0.95
    
    train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
    running_loss = 0.0
    
    for X, y in train_iter:
        y_hat = net(X.to(torch.float32))
        l = loss(y_hat,y).sum()#数据集损失函数的值=每个样本的损失函数值的和。            
        optimizer.zero_grad()#对优化函数梯度清零
        l.backward()#对损失函数求梯度
        optimizer.step() 
        
        running_loss += l.item()
        n += y.shape[0]
    #print(train_acc_sum/n)
    train_acc , train_pred=evaluate_accuracy(train_iter, net)
    test_acc , test_pred= evaluate_accuracy(test_iter, net)
    plot_test.append(test_acc)
    
    a,b,c = all_results(test_pred,y_true)
    plot_roc .append(a)
    plot_sensitivity .append(b)
    plot_specificity .append(c)
    
    plot_train.append(train_acc)
    accuracies.append(test_acc)
    losses.append(running_loss)
    print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f'
          % (epoch + 1, running_loss, train_acc , test_acc))
    pre_time = now_time
    now_time = time.time()
    print('该轮训练用时：',now_time-pre_time)

LinearNet(
  (linear1): Linear(in_features=186, out_features=32, bias=True)
  (relu): ReLU()
  (batchNorm1d1): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=32, out_features=2, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)
epoch 1, loss 0.6867, train acc 0.514, test acc 0.581
该轮训练用时： 0.008571863174438477
epoch 2, loss 0.6790, train acc 0.556, test acc 0.613
该轮训练用时： 0.005984067916870117
epoch 3, loss 0.6885, train acc 0.625, test acc 0.710
该轮训练用时： 0.006981611251831055
epoch 4, loss 0.6855, train acc 0.653, test acc 0.742
该轮训练用时： 0.007978677749633789
epoch 5, loss 0.6729, train acc 0.722, test acc 0.774
该轮训练用时： 0.005983591079711914
epoch 6, loss 0.6711, train acc 0.764, test acc 0.806
该轮训练用时： 0.005984067916870117
epoch 7, loss 0.6619, train acc 0.847, test acc 0.774
该轮训练用时： 0.0069828033447265625
epoch 8, loss 0.6676, train acc 0.861, test acc 0.806
该轮训练用时： 0.007977485656738281
epoch 9, loss 0.6542, train acc 0.83

In [12]:
# 使用交叉验证
from sklearn.model_selection import KFold

#x_train,x_test,y_train,y_test = train_test_split(df, label, test_size = 0.1)  


train = df
label == label

accuracy = []    
train_accuracy = []
mean_1 = []
mean_2 = []
mean_3 = []
k = 10
for i in range(k):
    kfold = KFold(n_splits=10,shuffle = True)
    print('第 %d 次交叉验证' %(i+1))
    test_acc_eve = []
    cnt = 1
    for train_index, test_index in kfold.split(train, label):

        #print(test_index)
        # train_index 就是分类的训练集的下标，test_index 就是分配的验证集的下标
        this_train_x, this_train_y = train.iloc[train_index], label.iloc[train_index]  # 本组训练集
        this_test_x, this_test_y = train.iloc[test_index], label.iloc[test_index]  # 本组验证集
        # 训练本组的数据，并计算准确率
        #model.fit(this_train_x, this_train_y)
        
        train_dataset = Data.TensorDataset(torch.tensor(this_train_x.values), torch.tensor(this_train_y.values))
        test_dataset = Data.TensorDataset(torch.tensor(this_test_x.values), torch.tensor(this_test_y.values))
        batch_size = 256
        num_workers = 0
        train_iter = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
        test_iter = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
        for X, y in test_iter:
            y_true = y
        net = LinearNet(num_inputs,num_outputs)
        
        num_epochs = 100
        now_time = time.time()
        losses = []
        plot_roc = []
        plot_sensitivity = []
        plot_specificity = []
        test_accuracies = []
        train_accuracies = []
        #损失函数和优化器
        loss = torch.nn.CrossEntropyLoss()
        LR = 0.1
        optimizer = torch.optim.SGD(
                                    net.parameters(), 
                                    lr = LR,
                                    weight_decay = 0.1
                                     )
        best_pred = []
        best_y = []
        best_test_acc = 0
        for epoch in range(num_epochs):

            if epoch % 10 == 0:
                for p in optimizer.param_groups:
                    p['lr'] *= 0.95

            train_l_sum, train_acc_sum, n = 0.0, 0.0, 0
            running_loss = 0.0

            for X, y in train_iter:
                y_hat = net(X.to(torch.float32))
                l = loss(y_hat,y).sum()#数据集损失函数的值=每个样本的损失函数值的和。            
                optimizer.zero_grad()#对优化函数梯度清零
                l.backward()#对损失函数求梯度
                optimizer.step() 

                running_loss += l.item()
                n += y.shape[0]
            #print(train_acc_sum/n)
            train_acc , train_pred=evaluate_accuracy(train_iter, net)
            test_acc , test_pred= evaluate_accuracy(test_iter, net)
            if test_acc>best_test_acc:
                best_test_acc = test_acc
            a,b,c = all_results(test_pred,y_true)
            plot_roc .append(a)
            plot_sensitivity .append(b)
            plot_specificity .append(c)    
                
            test_accuracies.append(test_acc)
            train_accuracies.append(train_acc)
            losses.append(running_loss)
            pre_time = now_time
            now_time = time.time()
        print('第 %d 折,train acc %.3f, test acc %.3f,test roc %.3f,test sensitivity %.3f,test specificity %.3f'
          % (cnt,np.array(train_accuracies).max() ,np.array(test_accuracies).max(),np.array(plot_roc).mean(),np.array( plot_sensitivity).mean(),np.array(plot_specificity).mean()))
        cnt = cnt+1
        test_acc_eve.append(np.array(test_accuracies).max())
        accuracy.append(np.array(test_accuracies).max())
        train_accuracy.append(np.array(train_accuracies).max())
        mean_1.append(np.array(plot_roc).mean())
        mean_2.append(np.array(plot_sensitivity).mean())
        mean_3.append(np.array(plot_specificity).mean())
    print(np.array(test_acc_eve).mean())
print('平均测试集准确率',np.array(accuracy).sum()/(k*10))
print('平均训练集准确率',np.array(train_accuracy).sum()/(k*10))
print('roc',np.array(mean_1).mean())
print('sensitivity',np.array(mean_2).mean())
print('specificity',np.array(mean_3).mean())

第 1 次交叉验证
第 1 折,train acc 0.924, test acc 0.909,test roc 0.777,test sensitivity 0.638,test specificity 0.917
第 2 折,train acc 0.946, test acc 0.818,test roc 0.636,test sensitivity 0.580,test specificity 0.693
第 3 折,train acc 0.935, test acc 0.909,test roc 0.854,test sensitivity 0.885,test specificity 0.822
第 4 折,train acc 0.935, test acc 1.000,test roc 0.896,test sensitivity 0.947,test specificity 0.846
第 5 折,train acc 0.914, test acc 1.000,test roc 0.754,test sensitivity 0.534,test specificity 0.975
第 6 折,train acc 0.925, test acc 1.000,test roc 0.835,test sensitivity 0.694,test specificity 0.977
第 7 折,train acc 0.946, test acc 0.800,test roc 0.722,test sensitivity 0.557,test specificity 0.887
第 8 折,train acc 0.957, test acc 0.900,test roc 0.769,test sensitivity 0.638,test specificity 0.900
第 9 折,train acc 0.946, test acc 0.900,test roc 0.864,test sensitivity 0.867,test specificity 0.861
第 10 折,train acc 0.925, test acc 1.000,test roc 0.860,test sensitivity 0.925,test specificity 0.795

第 2 折,train acc 0.935, test acc 0.909,test roc 0.810,test sensitivity 0.748,test specificity 0.872
第 3 折,train acc 0.946, test acc 0.909,test roc 0.781,test sensitivity 0.631,test specificity 0.930
第 4 折,train acc 0.914, test acc 1.000,test roc 0.909,test sensitivity 0.882,test specificity 0.936
第 5 折,train acc 0.935, test acc 1.000,test roc 0.836,test sensitivity 0.782,test specificity 0.890
第 6 折,train acc 0.925, test acc 1.000,test roc 0.876,test sensitivity 0.940,test specificity 0.813
第 7 折,train acc 0.935, test acc 0.800,test roc 0.722,test sensitivity 0.694,test specificity 0.750
第 8 折,train acc 0.935, test acc 0.700,test roc 0.589,test sensitivity 0.491,test specificity 0.687
第 9 折,train acc 0.946, test acc 0.800,test roc 0.642,test sensitivity 0.343,test specificity 0.942
第 10 折,train acc 0.935, test acc 1.000,test roc 0.953,test sensitivity 0.924,test specificity 0.982
0.8936363636363638
第 10 次交叉验证
第 1 折,train acc 0.935, test acc 0.909,test roc 0.806,test sensitivity 0.720,te